In [6]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import os

In [229]:
class Animal():
    def __init__(self, vars ,**kwargs):
        self.__dict__.update(vars, **kwargs)

class crawl(Animal):

    def ptt(self, url):
        header = self.__dict__
        r = requests.get(url, headers=header)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
       
            web_list = []
            for i in soup.find_all("div", class_='r-ent'):
                j = i.find_all("a", href=True)
                if len(j)>0:
                    web_list.append("https://www.ptt.cc/"+j[0]["href"])

        else: return "something wrong about ptt stock."

        for j in web_list:
            r2 = requests.get(j, headers=header)
            if r2.status_code == 200:

                soup2 = BeautifulSoup(r2.text, "html.parser")
                soup2.title.string.split("-")[0]

                text = soup2.find(id="main-container").text.split("--")
                pre_text = text[0].split('\n')

                push_text= []
                for i in soup2.find_all("div", class_="push"):
                    push_text.append(i.text)

                with open(os.path.join("news/ptt/", soup2.title.string.split("-")[0].replace(r'/',"")+".txt"), "w") as f:
                    f.write("\n".join(pre_text[1:]))
                    f.write("\n".join(push_text))

            else: return "th post %s has something wrong" %j
        



In [225]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
cookie = "uuid_tt_dd=10_20564905260-1703225841522-750867; c_first_ref=www.google.com; c_segment=3; c_adb=1; dc_sid=df9d26e5f60c9796335d2b2bdc46e0aa; hide_login=1; SidecHatdocDescBoxNum=true; creative_btn_mp=3; dc_session_id=10_1704181968889.661952; SESSION=ee7d1139-efc1-4491-8412-ffed93215f66; c_first_page=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_dsid=11_1704184443684.220868; c_pref=https%3A//www.google.com/; c_ref=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-102261567-blog-108477055.235%5Ev40%5Epc_relevant_anti_t3; c_utm_relevant_index=2; c_page_id=default; log_Id_pv=19; loginbox_strategy=%7B%22taskId%22%3A308%2C%22abCheckTime%22%3A1704163911128%2C%22version%22%3A%22exp1%22%2C%22blog-threeH-dialogtipShowTimes%22%3A8%2C%22blog-threeH-dialog%22%3A1704182063207%7D; log_Id_view=477; log_Id_click=7; dc_tos=s6mm2j"
x = dict({"User-Agent":user_agent, "Cookie":cookie})

In [230]:
test = crawl(x)
test.ptt(url="https://www.ptt.cc/bbs/Stock/index6815.html")

In [220]:
r = requests.get(r_ptt[0], headers=x)

soup = BeautifulSoup(r.text, "html.parser")
soup.title.string.split("-")[0]

text = soup.find(id="main-container").text.split("--")
pre_text = text[0].split('\n')

push_text= []
for i in soup.find_all("div", class_="push"):
    push_text.append(i.text)

with open(os.path.join("news/ptt/", soup.title.string.split("-")[0]+".txt"), "w") as f:
    f.write("\n".join(pre_text[1:]))
    f.write("\n".join(push_text))

In [228]:
'news/ptt/[閒聊] 2024/01/02 盤中閒聊 .txt'.replace(r'/',"")

'newsptt[閒聊] 20240102 盤中閒聊 .txt'